# Utils

In [ ]:
#| default_exp utils

In [ ]:
#| export
from fastai.vision.all import *

from similarity_learning.all import *


def fr_feature_extractors():
    return [cut_model_by_name(FaceNetInceptionResnetV1(pretrained, classify=False), 'last_bn')
            for pretrained in ['vggface2', 'casia-webface']]


def fr_dataloaders():
    return (LFWPeople().all().train.dl(),
            PinterestFaces(splitter=IndexSplitter([])).train.dl())


In [ ]:
#| export
from torch import Tensor

from fastai.vision.all import *


class ClassFeatures(Tensor):
    @classmethod
    def compute(cls, dl, fe, min_samples=0):
        fe.cuda().eval().requires_grad_(False)
        features, targets = L((fe(x), y) for x, y in progress_bar(dl)).zip().map(torch.cat)
        return {t.item(): cls(features[targets==t].as_subclass(Tensor).detach())
                for t in targets.unique() if features[targets==t].size(0) > min_samples}

    @property
    def centroid(self):
        return self.mean(0)

    @property
    def centroid_dir(self):
        return F.normalize(self.centroid, dim=0)

    @property
    def off_centroid_features(self):
        proj_mat = self.centroid_dir.outer(self.centroid_dir)
        return self - (self @ proj_mat)

In [ ]:
#| export
import seaborn as sns


def plot_hist(values, title=None, **kwargs):
    ax = sns.kdeplot(values, **kwargs)
    ax.set_title(title or f'$\mu={values.mean().round(decimals=2)}, \sigma={values.std().round(decimals=2)}$')
    return ax

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()